In [5]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from numpy.random import randn, rand
from scipy.stats import multivariate_normal
from tqdm import tqdm


In [6]:
def load_data_camera(K_csv, C_csv):
    """Loads data from the specified path"""
    K = pd.read_csv(K_csv, header=None).to_numpy()
    # print(pd.read_csv(K_csv))
    C = pd.read_csv(C_csv, header=None).to_numpy()
    return K, C

def load_measurement(meas_csv):
    """Loads data from the specified path"""
    meas = pd.read_csv(meas_csv,header=None).to_numpy()
    return meas

def load_transformations(R_csv, T_csv):
    """Loads data from the specified path"""
    R = pd.read_csv(R_csv,header=None).to_numpy()
    T = pd.read_csv(T_csv,header=None).to_numpy()
    return R, T

In [7]:
# Camera 1
K1, C1 = load_data_camera('data_csv/Kf_1.csv', 'data_csv/C_1.csv')
z1 = load_measurement('data_csv/z_1.csv')

# Camera 2
K2, C2 = load_data_camera('data_csv/Kf_2.csv', 'data_csv/C_2.csv')
z2 = load_measurement('data_csv/z_2.csv')

# load the transformations
R, T = load_transformations('data_csv/R.csv', 'data_csv/T.csv')

In [8]:
class PF:
    def __init__(self, N, Q, process_func):
        
        self.N = N
        x = np.zeros((N, 3))
        self.x = x #x + np.random.multivariate_normal(np.zeros(3), Q, size=N)
        self.w = np.ones(N) / N

        self.f = process_func
        self.Q = Q


    def predict(self):
        pass

    def sample_motion(self):
        # f = 1
        # x = f * x
        # x = np.random.multivariate_normal(self.x, self.Q)
        x = self.f(self.x, self.Q)
        self.x = x

    def importance_sampling(self, z, h_func, R):
        for i in range(self.N):
            w = multivariate_normal.pdf(z, h_func(self.x[i]), R)
            self.w[i] *= w

        self.w = self.w / np.sum(self.w)
        n_eff = 1 / np.sum(np.power(self.w, 2))

        if n_eff < self.N:
            self.resample()
        return self.x

    def resample(self):
        # low variance resampling
        W = np.cumsum(self.w)
        r = rand(1) / self.N
        # r = 0.5 / self.n
        j = 1
        for i in range(self.N):
            u = r + (i - 1) / self.N
            while u > W[j]:
                j = j + 1
            self.x[i, :] = self.x[j, :]
            self.w[i] = 1 / self.N




In [43]:
P1 = np.array([1, 1, 1])

Q1 = np.eye(3) * 10
Q2 = np.eye(3) * 0

R1 = np.eye(2) * 1000
R2 = np.eye(2) * 1000

def h1_func(x):
    pred_z = (1/x[2]) * K1 @ x[:2].reshape(-1,1) + C1
    return pred_z.reshape(-1,)

def h2_func(x):
    trans_P = (R.T@ x - R.T@T).reshape(-1,)
    # print(trans_P.shape)
    pred_z = (1/trans_P[2]) * K2 @ trans_P[:2].reshape(-1,1) + C2
    return pred_z.reshape(-1,)

def process_func(x, Q):
    x = x.reshape(-1,3)
    # print(x.shape)
    return x + np.random.multivariate_normal(np.zeros(3), Q, size=x.shape[0])

particles_size = 10000
particle_filter = PF(particles_size, Q1, process_func)

In [44]:
# testing functions:
# x = np.array([1, 1, 1])
# x = np.array([-0.14477037, 0.8104404, 8.6541336 ])
# x = np.array([-0.14477037, 0.8104404, 8.6541336 ])

print(h1_func(x))
print(h2_func(x))
print(process_func(x, Q1))
print(process_func(x, Q2))

# print(np.random.multivariate_normal(np.zeros(3), Q1, size=1000).shape)

z =  z1[1]
print(z.shape)
print(z)
print(h1_func(x))
print(R1)
multivariate_normal.pdf(z, h1_func(x).reshape(-1,), R1)


[835.9 771.8]
[259.66796956 760.43129513]
[[-1.15556637  0.18585733 -2.2136541 ]]
[[1. 1. 1.]]
(2,)
[386.9 302.4]
[835.9 771.8]
[[1000.    0.]
 [   0. 1000.]]


3.795653445914226e-96

# Sequence

In [45]:

for z_1, z_2 in tqdm(zip(z1, z2)):
    particle_filter.sample_motion()
    particle_filter.importance_sampling(z_1, h1_func, R1)
    # w = multivariate_normal.pdf(z_1, h1_func(x), R1)
    # print(w)
    particle_filter.importance_sampling(z_2, h2_func, R2)
    print(particle_filter.x.mean(axis=0))

1it [00:01,  1.25s/it]

[-0.19911757  0.75427719  3.49147291]


2it [00:02,  1.23s/it]

[-0.20454954  0.28069524  4.07379932]


3it [00:03,  1.23s/it]

...

[-0.22621516  0.54610363  3.56482837]


19it [00:23,  1.23s/it]

[-0.01421802  0.8273513   3.83370223]


20it [00:24,  1.22s/it]

[-0.1539303   0.48638049  3.86667759]


In [46]:
# output
particle_filter.x.mean(axis=0)

array([-0.1539303 ,  0.48638049,  3.86667759])

# Batch 

In [47]:
def h_func(x):
    pred_z1 = h1_func(x) 
    pred_z2 = h2_func(x)
    return np.hstack((pred_z1, pred_z2))

In [51]:
R_stacked = np.block([[R1, np.zeros((2,2))], [np.zeros((2,2)), R2]])
x = np.array([1, 1, 1])
for z_1, z_2 in tqdm(zip(z1, z2)):
    particle_filter.sample_motion()
    z = np.hstack((z_1, z_2))#.reshape(-1,)
    # w = multivariate_normal.pdf(z, h_func(x), R_stacked)
    # print(w)
    # z = np.hstack((z_1, z_2)).reshape(-1,1)
    # print(z, z_1, z_2)
    particle_filter.importance_sampling(z, h_func, R_stacked)


20it [00:15,  1.30it/s]


In [52]:
particle_filter.x.mean(axis=0)

array([-0.58289359,  0.85425295,  3.6565127 ])